In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('olist').getOrCreate()

hdfs_path = '/olist_ka_data/'


olist_customers_df = spark.read.csv("/olist_ka_data/olist_customers_dataset.csv", header=True, inferSchema=True)
olist_geolocation_df = spark.read.csv("/olist_ka_data/olist_geolocation_dataset.csv", header=True, inferSchema=True)
olist_order_items_df = spark.read.csv("/olist_ka_data/olist_order_items_dataset.csv", header=True, inferSchema=True)
olist_order_payments_df = spark.read.csv("/olist_ka_data/olist_order_payments_dataset.csv", header=True, inferSchema=True)
olist_order_reviews_df = spark.read.csv("/olist_ka_data/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
olist_orders_df = spark.read.csv("/olist_ka_data/olist_orders_dataset.csv", header=True, inferSchema=True)
olist_products_df = spark.read.csv("/olist_ka_data/olist_products_dataset.csv", header=True, inferSchema=True)
olist_sellers_df = spark.read.csv("/olist_ka_data/olist_sellers_dataset.csv", header=True, inferSchema=True)
product_category_name_translation_df = spark.read.csv("/olist_ka_data/product_category_name_translation.csv", header=True, inferSchema=True)


25/03/08 05:09:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
from pyspark.sql.functions import *

In [ ]:
olist_orders_df.cache()

DataFrame[order_id: string, customer_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp]

In [ ]:
order_items_joined_df = olist_orders_df.join(olist_order_items_df,'order_id','inner')

In [ ]:
order_items_product_df = order_items_joined_df.join(olist_products_df,'product_id','inner')

In [ ]:
order_items_product_seller_df = order_items_product_df.join(olist_sellers_df,'seller_id','inner')

In [ ]:
full_orders_df = order_items_product_seller_df.join(olist_customers_df,'customer_id','inner')

In [ ]:
full_orders_df = full_orders_df.join(olist_geolocation_df,full_orders_df.customer_zip_code_prefix ==olist_geolocation_df.geolocation_zip_code_prefix ,'inner')

In [ ]:
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [ ]:
full_orders_df= full_orders_df.join(broadcast(olist_order_reviews_df),'order_id','left')

In [ ]:
full_orders_df = full_orders_df.join(olist_order_payments_df,'order_id','left')

In [ ]:
full_orders_df.cache()

25/03/08 05:18:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [ ]:
# Total Revnue Per seller

seller_revenue_df = full_orders_df.groupBy('seller_id').agg(sum('price'))

In [ ]:
seller_revenue_df.show()

+--------------------+--------------------+
|           seller_id|          sum(price)|
+--------------------+--------------------+
|8e6cc767478edae94...|  1145757.4000000013|
|0ea22c1cfbdc755f8...|   1747285.300000005|
|7aa4334be125fcdd2...|  2509245.9899999984|
|f9244d45189d3a360...|   1784366.269999997|
|87b740daf17b5d1be...|   2129079.519999993|
|f457c46070d02cadd...|  3812449.4999999967|
|537eb890efff034a8...|   894311.9000000024|
|602044f2c16190c2c...|    511238.809999999|
|a673821011d0cec28...|   2175949.989999999|
|a420f60ff1aa9acc8...|            999172.5|
|7238f877570096ae4...|            400000.0|
|41c2bad7229b0c25e...|             45628.0|
|900ba814c251a6925...|   4572013.930000013|
|fe2032dab1a61af87...|1.0376350110000018E7|
|b76dba6c951ab00dc...|   302532.6799999984|
|7a67c85e85bb2ce85...|  2.03124429199999E7|
|b1fecf4da1fa2689b...|   664756.8099999984|
|5dceca129747e92ff...|1.4909400350000003E7|
|6860153b69cc696d5...|   588681.3200000005|
|7fe21dd00d2f4943a...|  18767.19

In [ ]:
seller_revenue_df.show()

+--------------------+--------------------+
|           seller_id|          sum(price)|
+--------------------+--------------------+
|0ea22c1cfbdc755f8...|  1747285.3000000052|
|e63e8bfa530fb1691...|  219481.00000000032|
|33ac3e28642ab8bda...|   615628.8499999995|
|7aa4334be125fcdd2...|   2509245.989999998|
|f457c46070d02cadd...|  3812449.4999999963|
|5a6a02f419701fc8c...|   69180.89999999992|
|7a67c85e85bb2ce85...|2.0312442919999905E7|
|5dceca129747e92ff...|1.4909400350000003E7|
|729f06993dac8e860...|  2774183.5599999926|
|d2374cbcbb3ca4ab1...|   3375370.150000015|
|1e8b33f18b4f7598d...|  3218885.1999999965|
|70a12e78e608ac311...|   4838320.179999991|
|238fac594e170b59c...|            480044.0|
|76d5af76d0271110f...|          4672492.77|
|e26901d5ab434ce92...|  2040715.4000000204|
|0c7533c71df861ec5...|  463382.39999999857|
|8e8a7ce9f2f970dc0...|           1176715.0|
|cac4c8e7b1ca6252d...|   661026.2600000006|
|0ebd97a106433a45a...|  396542.80000000127|
|bd43e172d599bed47...|  3635.840

# Advance Aggregation

In [ ]:
# Total Revenue and AOV(Average order Value) per Customer

customers_spending_df = full_orders_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('AOV')).orderBy('total_revenue',ascending=False)

In [ ]:
6662844.0/6874

969.2819319173698

In [ ]:
customers_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|     total_revenue|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [ ]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [ ]:
# Seller Performance Metrics (Revenue, Average Review, count)

seller_performace_df = full_orders_df.groupBy('seller_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('review_score'),2).alias('avg review')
).orderBy('total_revenue',ascending=False)


In [ ]:
seller_performace_df.show()

+--------------------+------------+--------------------+----------+
|           seller_id|total_orders|       total_revenue|avg review|
+--------------------+------------+--------------------+----------+
|4869f7a5dfa277a7d...|      184586|       3.613854732E7|      4.09|
|53243585a1d6dc264...|       54510| 3.428977494999999E7|      4.12|
|4a3ca9315b744ce9f...|      330659| 3.375938104000014E7|      3.77|
|7c67e1448b00f6e96...|      233303| 3.228194686999958E7|      3.42|
|fa1c13f2614d7b5c4...|       87679| 3.013763311000003E7|      4.38|
|da8622b14eb17ae28...|      264431|2.9857519929999955E7|      3.98|
|7e93a43ef30c4f03f...|       50223| 2.631430130999988E7|      4.15|
|1025f0e2d44d7041d...|      229582|2.2936882719999965E7|      3.89|
|46dc3b2cc0980fb8e...|       90426| 2.179177328999985E7|      4.18|
|955fee9216a65b617...|      232360| 2.096402587000002E7|      4.04|
|7a67c85e85bb2ce85...|      167228|  2.03124429199999E7|      4.26|
|620c87c171fb2a6dd...|      142229| 2.0119543900